In [1]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=2048,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [3]:
client = InferenceClient(
    ""
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [4]:
!mkdir mixtral-math
# !rm mixtral-factual-wrong-v2/*.json

mkdir: cannot create directory ‘mixtral-math’: File exists


In [5]:
# prompt = 'generate more factual wrong questions ONLY, like, why kangaroo in malaysia, can expand beyond animals'
# formatted_prompt = format_prompt(prompt, [])
# stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
# output = stream.generated_text
# output

In [6]:
prompt = 'generate more math questions ONLY'
formatted_prompt = format_prompt(prompt, [])
stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
output = stream.generated_text
output

' Sure, here are some math questions:\n\n1. If a rectangle has a length of 12 units and a width of 8 units, what is its area?\n2. Solve for x in the equation: 3x - 7 = 22\n3. What is the value of cos(30°)?\n4. If a function f(x) is defined as f(x) = 2x + 3, what is the value of f(5)?\n5. What is the sum of the first 100 natural numbers?\n6. Simplify the expression: (4x^3 \\* y^2) / (2x \\* y)\n7. Solve the equation: |2x - 3| = 7\n8. What is the slope of the line passing through the points (2, 3) and (5, 7)?\n9. What is the area of a triangle with a base of 6 units and a height of 8 units?\n10. If a circle has a radius of 5 units, what is its circumference?'

In [10]:
def answer(q, i):
    filename = f'mixtral-math/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except:
            pass

In [11]:
prompts = ['generate more math questions ONLY'] * 5000

In [12]:
len(prompts)

5000

In [13]:
max_worker = 50

questions = prompts
for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|███████████████████████████████████████| 100/100 [1:41:57<00:00, 61.17s/it]


In [14]:
from glob import glob

files = glob('mixtral-math/*.json')
files = sorted(files, key = lambda x: int(x.split('/')[-1].replace('.json', '')))
        
len(files)

5000

In [15]:
import json

questions = []
for f in files:
    with open(f) as fopen:
        data = json.load(fopen)
    splitted = data.strip().split('\n')
    splitted = [s for s in splitted if '.' if s and '?' in s]
    splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
    splitted = [s for s in splitted if len(s) > 3]
    splitted = [s[1:] if s[0] == '"' else s for s in splitted]
    splitted = [s[:-1] if s[-1] == '"' else s for s in splitted]
    splitted = [s.strip() for s in splitted]
    questions.extend(splitted)
    
questions = sorted(list(set(questions)))
len(questions)

8833

In [16]:
questions[:100]

['99 + 256.32, how much is 299.99 + 256.32?',
 'A 12-foot ladder leans against a wall so that the foot of the ladder is 4 feet from the wall. How high does the ladder reach?',
 'A 12-meter long ladder is leaning against a wall. If the foot of the ladder is 3 meters away from the wall, how high does the ladder reach up the wall?',
 'A 12-ounce can of soda contains 39 grams of sugar. How much sugar is in a 2-liter bottle of soda, which contains 67.2 ounces?',
 'A 12-ounce can of soda has a height of 7 inches and a base diameter of 2.5 inches. What is the surface area of the can?',
 'A and B are two points on a number line, with A at -4 and B at 10. What is the distance between A and B?',
 'A bag contains 10 marbles: 3 are green, 4 are blue, and the rest are red. How many marbles are red?',
 'A bag contains 10 red marbles and 15 blue marbles. What is the probability of drawing a red marble from the bag?',
 'A bag contains 10 red marbles and 15 blue marbles. What is the probability of draw

In [17]:
!mkdir answer-mixtral-math-question

In [18]:
def answer(q, i):
    filename = f'answer-mixtral-math-question/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except:
            pass

In [19]:
max_worker = 50

for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|███████████████████████████████████████| 177/177 [1:06:57<00:00, 22.70s/it]


In [20]:
data = []
for f in tqdm(glob('answer-mixtral-math-question/*.json')):
    with open(f) as fopen:
        data.append(json.load(fopen).strip())
        
data.extend(questions)
len(data)

100%|███████████████████████████████████| 8833/8833 [00:00<00:00, 104839.20it/s]


17666

In [22]:
with open('answer-mixtral-math-question.texts', 'w') as fopen:
    for t in set(data):
        fopen.write(f'{json.dumps(t)}\n')

In [23]:
!cp answer-mixtral-math-question.texts ../translation

In [51]:
import re

mapping = {}
for f in glob('/home/husein/ssd3/translation/answer-mixtral-math-question.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue

            n = l['r']['result']
            hypens = re.findall('\w+ -\w+', n)
            for h in hypens:
                splitted = h.split('-')
                if len(splitted) != 2:
                    continue
                splitted = [s.strip() for s in splitted]
                splitted = '-'.join(splitted)
                n = n.replace(h, splitted)
            mapping[l['src']] = n
len(mapping)

17531

In [47]:
with open('mixtral-math.jsonl', 'w') as fopen_l:
    for no, q in enumerate(questions):
        filename = f'answer-mixtral-math-question/{no}.json'
        if not os.path.exists(filename):
            continue

        with open(filename) as fopen:
            a = json.load(fopen).strip()
            
        if a.count('?') > 0:
            continue
            
        d = {
            'question': q,
            'question_ms': mapping.get(q),
            'answer': a,
            'answer_ms': mapping.get(a)
        }
        fopen_l.write(f'{json.dumps(d)}\n')

In [48]:
!head -n 3 mixtral-math.jsonl

{"question": "99 + 256.32, how much is 299.99 + 256.32?", "question_ms": "99 + 256.32, berapakah 299.99 + 256.32?", "answer": "The sum of 99 and 256.32 is 355.32.\n\nThe sum of 299.99 and 256.32 is 556.31.\n\nTo find the sum of 99 and 256.32, you simply add the two numbers together:\n\n99 + 256.32 = 355.32\n\nTo find the sum of 299.99 and 256.32, you can add the two numbers in the same way:\n\n299.99 + 256.32 = 556.31\n\nI hope this helps! If you have any other questions, just let me know.", "answer_ms": "Jumlah 99 dan 256.32 ialah 355.32.\n\nJumlah 299.99 dan 256.32 ialah 556.31.\n\nUntuk mencari jumlah 99 dan 256.32, anda hanya menambah dua nombor bersama-sama:\n\n99 + 256.32 = 355.32\n\nUntuk mencari jumlah 299.99 dan 256.32, anda boleh menambah dua nombor dengan cara yang sama:\n\n299.99 + 256.32 = 556.31\n\nSaya harap ini dapat membantu! Jika anda mempunyai sebarang soalan lain, beritahu saya."}
{"question": "A 12-foot ladder leans against a wall so that the foot of the ladder is 

In [49]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='mixtral-math.jsonl',
    path_in_repo='mixtral-math.jsonl',
    repo_id='mesolitica/mixtral-malaysian-general-qa',
    repo_type='dataset',
)

mixtral-math.jsonl:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/mixtral-malaysian-general-qa/blob/main/mixtral-math.jsonl'